1) Exemplo introdutório carro a combustão

In [1]:
def car (env):
    while True:
        print(f'Começou a estacionar em %ds' % env.now)
        tempo_estacionar = 10
        yield env.timeout(tempo_estacionar)
        print(f'Começou a dirigir em %ds' % env.now)
        tempo_dirigindo = 3
        yield env.timeout(tempo_dirigindo)

In [2]:
import simpy
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Começou a estacionar em 0s
Começou a dirigir em 10s
Começou a estacionar em 13s


2) Exemplo introdutório com carro elétrico

*object* é o argumento que está sendo passado na classe e o parâmetro *self* na função() irá receber essa instância.

In [3]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Começa o processo run sempre que uma instância for criada.
        self.action = env.process(self.run())

    def run (self):
        while True:
            print('Estacionou e começou a carregar em %d s' % self.env.now)
            tempo_carregando = 5
            #Produzimos o processo que process () retorna 
            #para aguardar o término
            yield self.env.process(self.charge(tempo_carregando))

            #O processo de carregamento terminou e 
            #podemos dirigir novamente
            print('Começou a dirigir em %d s' %self.env.now)
            tempo_dirigindo = 2
            yield self.env.timeout(tempo_dirigindo)

    def charge (self, duration):
        yield self.env.timeout(duration)
            

In [4]:
import simpy
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Estacionou e começou a carregar em 0 s
Começou a dirigir em 5 s
Estacionou e começou a carregar em 7 s
Começou a dirigir em 12 s
Estacionou e começou a carregar em 14 s


2.1) Interromper o processo: (Interrupt)

In [5]:
def driver (env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [6]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())

    def run (self):
        while True:
            print('Estacionou e começou a carregar em %d s' % self.env.now)
            tempo_carregando = 5
            #O carregamento da bateria pode ser interrompido
            try:
                yield self.env.process(self.charge(tempo_carregando))
            except simpy.Interrupt:
                #Quando recebe o Interrupt, para de carregar
                # e troca para o estado "dirigindo"
                print('Foi interrompido. Espero que tenha carga suficiente')
          
            print('Começou a dirigir em %d s' %self.env.now)
            tempo_dirigindo = 2
            yield self.env.timeout(tempo_dirigindo)

    def charge (self, duration):
        yield self.env.timeout(duration)

In [7]:
import simpy
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Estacionou e começou a carregar em 0 s
Foi interrompido. Espero que tenha carga suficiente
Começou a dirigir em 3 s
Estacionou e começou a carregar em 5 s
Começou a dirigir em 10 s
Estacionou e começou a carregar em 12 s


3) Recursos compartilhados: (Resource)

In [56]:
def car (env, nome, bcs, tempo_dirigindo, tempo_carregando):
    # Simula dirigindo até o posto de carregamento (bcs)
    yield env.timeout(tempo_dirigindo)

    # Solicita (request) uma das estações de carregamento
    print('> %s chega em %d s' % (nome, env.now))
    with bcs.request() as req:
        yield req

        # Carrega a bateria
        print('>> %s começou a carregar em %s s' % (nome, env.now))
        yield env.timeout (tempo_carregando)
        print ('>>> %s saindo da estação em %d s' % (nome, env.now))


Um recurso (resource) precisa referenciar a um Environment e uma capacidade quando é criado:

In [57]:
import simpy
env = simpy.Environment()
bcs = simpy.Resource(env, capacity = 2)

Agora, pode-se criar o processo "car" e passar nosso recurso, além de outros parâmetros:

In [60]:
donos = ['Luccas', 'Victor', 'Giglio', 'Ernani']

for i in range(len(donos)):
    env.process(car(env, 'Carro do %s' % donos[i], bcs, i*2, 5))

In [61]:
# Rodamos a simulação:
env.run()

> Carro do Luccas chega em 12 s
>> Carro do Luccas começou a carregar em 12 s
> Carro do Victor chega em 14 s
>> Carro do Victor começou a carregar em 14 s
> Carro do Giglio chega em 16 s
>>> Carro do Luccas saindo da estação em 17 s
>> Carro do Giglio começou a carregar em 17 s
> Carro do Ernani chega em 18 s
>>> Carro do Victor saindo da estação em 19 s
>> Carro do Ernani começou a carregar em 19 s
>>> Carro do Giglio saindo da estação em 22 s
>>> Carro do Ernani saindo da estação em 24 s
